In [76]:
import math
import numpy as np
import pandas as pd 
from bs4 import BeautifulSoup
import requests
import urllib

In [7]:
""" 
!pip install beautifulsoup4
"""

In [154]:
"""
Function which takes input of list of tuples.
Tuples consists of name of food (string), and its proability (float). 
Outputs dictionary for recipes for top 3 foods and ingredients.
"""

foods = [('pizza', 0.951),('garlic bread', 0.013),('Grilled cheese sandwich', 0.004)]

def findRecipe(foods):
    print('Does this food have a main ingredient? (ie. input "Chicken" for Chicken burrito)')
    added = input('input "no" if it does not have one')
    if added == 'no':
        extra = ""
    else:
        extra = added
    outputs = {}
    for food in foods:
        
        output = []
        food_name, food_prob = food
        
        ##get top hit from allrecipes.com using food as query 
        query = extra + food_name 
        query = urllib.parse.quote_plus(query)

        r = requests.get('https://www.allrecipes.com/search/results/?search={}'.format(query))
        soup = BeautifulSoup(r.text, "html.parser")

        for item in soup.find_all('a', href=True):
            if "https://www.allrecipes.com/recipe/" in item['href']:
                top_link = item['href']
                break
        #get recipe and ingredients 
        r = requests.get(top_link)
        soup = BeautifulSoup(r.text, "html.parser")
        
        ##finding ingredients 
        ingredients = []
        for item in soup.find_all('span', class_= 'ingredients-item-name elementFont__body'):
            ingredients.append(str(item)[54:-8])
            
        ##finding recipe 
        counter = 1
        recipe = {}
        for item in soup.find_all("div" , class_='section-body elementFont__body--paragraphWithin elementFont__body--linkWithin'):
            recipe[counter] = str(item.p)[3:-5]
            counter += 1 
        
        
        ##append to output
        output.append(food_prob)
        output.append(ingredients)
        output.append(recipe)
        output.append(top_link)
        outputs[food_name] = output 
        
    return outputs 

In [155]:
"""
This is a function which shows the results of the searched query 
Takes input of a dictionary with keys of predicted foods and 
values of probability, ingredients, recipes, and the link in that order
Output is clean
"""
def cleanoutput(d):
    foods = d.keys()
    index = 1
    for food in foods:
        prob, ing, rec, link = d[food]
        print("The result ranked number {} was {} with probability of {} \n".format(index, food, prob))
        print("The ingredients required to cook this are:")
        for ings in ing:
            print(ings)
        print('\nThe Recipe for a dish like this would be:')
        for step in rec.keys():
            print("step {}: {}".format(step, rec[step]))
        print("\nSource: {}".format(link))
        print('-------')
        index += 1 

In [157]:
cleanoutput(findRecipe(foods))

Does this food have a main ingredient? (ie. input "Chicken" for Chicken burrito)
input "no" if it does not have onePepperoni
The result ranked number 1 was pizza with probability of 0.951 

The ingredients required to cook this are:
½ (12 ounce) can CONTADINA® Tomato Paste
1 teaspoon dried oregano, crushed
1 teaspoon dried basil, crushed
½ teaspoon garlic powder
½ teaspoon onion powder
½ teaspoon sugar
½ teaspoon salt
¼ teaspoon black pepper
3 ¼ cups all-purpose flour, or more as needed
2 (.25 ounce) envelopes FLEISCHMANN'S® Pizza Crust Yeast or RapidRise® Yeast
1 tablespoon sugar
1 ½ teaspoons salt
1 ⅓ cups very warm water (120 degrees F to 130 degrees F)
⅓ cup oil
1 (6 ounce) package HORMEL® Pepperoni
1 cup shredded mozzarella cheese, or more to taste

The Recipe for a dish like this would be:
step 1: For sauce: Combine all sauce ingredients with 1/2 cup water in a medium bowl; set aside for flavors to develop while making crust. Freeze remaining paste 
step 2: For crusts: Combine 2 